In [1]:
from src.utilities_rejection import RejectionReasonLabeling
from src.data_local import MergedDataPreprocessing
from cashe import get_cashed_input

In [2]:
# !pip install tensorflow --upgrade
df = get_cashed_input()

In [3]:
df

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,Episode_Key,ICD10,Diagnosis_Key
397551,10335,2024-01-01 09:58:15,194131,575.00,Routine preoperative anaes assessment,1.0,500.00,500.00,0.00,500.00,...,FILE,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
397548,10335,2024-01-01 09:58:15,194131,312.80,Cross Matching,1.0,340.00,340.00,0.00,340.00,...,FILE,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
397549,10335,2024-01-01 09:58:15,194131,11.38,TRANDATE INJ. 100 MG.-AMP.20ML,1.0,11.98,11.98,0.00,11.98,...,FILE,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
397550,10335,2024-01-01 09:58:15,194131,15346.75,Elective classical caesarean section,1.0,15700.00,15700.00,0.00,15700.00,...,FILE,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
397559,10335,2024-01-01 10:00:43,194132,0.88,"Hypodermic needle, single-use",1.0,1.10,1.10,0.00,1.10,...,FILE,N,H,2023-12-07 14:09:32,2023-12-09 09:52:28,2023-12-01,2023-12-10,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388637,9643,2024-04-30 12:13:09,238137,50.57,Peripheral intravenous cannula,1.0,64.50,64.50,9.03,55.47,...,FILE,N,H,2024-04-05 00:00:00,2024-04-19 00:00:00,2024-04-01,2024-04-10,1_150587-17,N23,1_10169499
388638,9643,2024-04-30 12:13:09,238137,555.18,Ultrasound of pelvis,1.0,689.80,689.80,0.75,689.05,...,FILE,N,H,2024-04-05 00:00:00,2024-04-19 00:00:00,2024-04-01,2024-04-10,1_150587-17,N23,1_10169499
388639,9643,2024-04-30 12:13:09,238137,45.65,Urinalysis Test,1.0,58.23,58.23,8.15,50.08,...,FILE,N,H,2024-04-05 00:00:00,2024-04-19 00:00:00,2024-04-01,2024-04-10,1_150587-17,N23,1_10169499
388640,9643,2024-04-30 12:13:09,238137,4.67,PANADREX 500 MG 24 TAB,24.0,5.05,5.05,0.93,4.12,...,FILE,N,H,2024-04-05 00:00:00,2024-04-19 00:00:00,2024-04-01,2024-04-10,1_150587-17,N23,1_10169499


# Rejection Reason Notes Codes Analysis

In [4]:
'''
class RejectionReasonLabeling:
    def __init__(self, df):
        self.df = df
        self.df['NPHIES_CODE'] = None

    def check_doctor_code(self):
        df = self.df

        label_technical = []
        for visit_id in set(df['VISIT_ID']):
            df_visit = df[df['VISIT_ID']==visit_id]
            if len(df_visit['DOCTOR_CODE'].value_counts())!=1:
                technical_issue = ['DOCTOR_CODE']* len(df_visit)
            else:
                technical_issue = [None]* len(df_visit)
            label_technical = label_technical + technical_issue
        df['TECHNICAL_ISSUE'] = label_technical

        return df
'''
print()

In [5]:
label_technical = RejectionReasonLabeling(df)
df = label_technical.recoginze_label()

In [ ]:
## CV-x-x problems are related to the provider contract mainly, which can't be a part of our predictive analytics

In [6]:
## MN-1-1 Service is not clinically justified based on clinical practice guideline (Service vs Diagnosis)
df[df['NPHIES_CODE']=='MN-1-1'][['SERVICE_DESCRIPTION','ICD10']]
## DERMATIN SOLUTION with ICD10 (H60) indicating Otitis externa ( Drug is incompatible with icd10 ).
## DERMATIN for foot itches, and otitis externa could be treated with hydrocortisone 3 drops to the affected ear daily they are two different things.

,SERVICE_DESCRIPTION,ICD10
397397,DERMATIN SOLUTION 1%,H60
397398,DEXAFLOX OPHTHALMIC SUSPENSION,H60
397468,OPTIDEX-T STERILE OPHTALMIC SUSP,H00.1
397606,PARACETOL 10 MG/ML 100 ML BOTTLE (PSI),H66.9
397541,PANADOL ADVANCE 500MG FILM COATED TABLET,H66.9
...,...,...
388372,RIAZOLE,Z39.00
388376,CRP,P51.9
388377,Automated complete Blood cell and differentia...,P51.9
388378,AVOBAN 2% W-W OINTMENT,P51.9


In [97]:
## SE-1-6: Investigation result is inadequate or missing
df[df['NPHIES_CODE']=='SE-1-6'][['SERVICE_DESCRIPTION','PATIENT_NO', 'PATIENT_ID', 'MEMBERSHIP_NO', 'PATIENT_ID_TYPE', 'PATIENT_NATIONAL_IQAMA_ID', 'PATIENT_DOB', 'PATIENT_NATIONALITY',  'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'PATIENT_MOBILE_NO',   'PATIENT_BADGE_NO', 'HIS_INSURANCE_CODE', 'POLICY_HOLDER_NO']]

,SERVICE_DESCRIPTION,PATIENT_NO,PATIENT_ID,MEMBERSHIP_NO,PATIENT_ID_TYPE,PATIENT_NATIONAL_IQAMA_ID,PATIENT_DOB,PATIENT_NATIONALITY,PATIENT_MARITAL_STATUS,PATIENT_GENDER,PATIENT_MOBILE_NO,PATIENT_BADGE_NO,HIS_INSURANCE_CODE,POLICY_HOLDER_NO
400859,"Radiography of teeth, full mouth",876090,1020440,1079611,1.0,1195216146,1996-10-16,SAUDI ARABIAN,S,F,551444436,1079611,275,1116797745
401581,"Radiography of teeth, full mouth",1182339,1345812,5291507,1.0,1084294865,1989-02-27,SAUDI ARABIAN,M,F,566102603,5291507,275,GRH/15163579
402709,"Radiography of teeth, full mouth",124165,51877,1417318,1.0,1003238548,1948-05-09,SAUDI ARABIAN,M,M,555644665,1417318,275,14888432
400522,"Radiography of teeth, full mouth",1014161,1167940,635153,2.0,2037303993,1983-10-27,YEMENI,M,M,500211499,635153,275,GRH/15218233-0
400537,"Radiography of teeth, full mouth",1044181,1199407,635156,2.0,2223614682,2005-07-17,YEMENI,S,M,500211499,635156,275,2037303993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325911,LAMICTAL 100 MG 30 TAB,137825,176890,54925667,1.0,1009343847,1973-07-30,SAUDI ARABIAN,M,M,503627060,54925667,2622,2230000326
325914,OLFEN PATCHES,137825,176890,54925667,1.0,1009343847,1973-07-30,SAUDI ARABIAN,M,M,503627060,54925667,2622,2230000326
325912,NEXIUM 20MG TAB BOX/28,137825,176890,54925667,1.0,1009343847,1973-07-30,SAUDI ARABIAN,M,M,503627060,54925667,2622,2230000326
339352,M-mode 2D real time u/s of heart,1196636,1361172,6755132,2.0,2008467603,1955-07-01,SUDANESE,M,M,507439462,6755132,183,185182033


In [100]:
## SE-1-8,9: Treatment plan is inadequate or missing, Quantity of service/procedure is inappropriate or missing
df[df['NPHIES_CODE'].isin(['SE-1-9','SE-1-8'])][['QTY', 'LINE_CLAIMED_AMOUNT', 'LINE_CLAIMED_AMOUNT_SAR', 'CO_PAY', 'CO_INSURANCE', 'LINE_ITEM_DISCOUNT', 'NET_AMOUNT', 'NET_VAT_AMOUNT']]
## some items quantities are way too many

,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT
404193,1.0,750.0,750.0,0.0,750.0,0.00,750.00,112.50
404200,1.0,135.0,135.0,0.0,135.0,33.75,101.25,15.19
403452,2.0,16.0,16.0,0.0,16.0,0.80,15.20,0.00
403475,2.0,1160.0,1160.0,0.0,1160.0,290.00,870.00,130.50
404226,1.0,5.4,5.4,0.0,5.4,1.35,4.05,0.00
...,...,...,...,...,...,...,...,...
229082,2.0,3380.0,3380.0,0.0,3380.0,0.00,3380.00,507.00
229083,1.0,810.0,810.0,0.0,810.0,202.50,607.50,91.13
229446,2.0,2070.0,2070.0,0.0,2070.0,0.00,2070.00,310.50
229444,1.0,1820.0,1820.0,0.0,1820.0,0.00,1820.00,273.00


In [103]:
## BE-1-1 : Co-pay was not collected from member
df[df['NPHIES_CODE'].isin(['BE-1-1'])][['QTY', 'CO_PAY', 'CO_INSURANCE', 'LINE_ITEM_DISCOUNT', 'NET_AMOUNT']]
## Most of CO_PAY values are zeros. (MOST)

,QTY,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT
407021,7.0,13.02,108.29,6.07,102.22
477325,14.0,8.47,36.13,2.23,33.90
407392,1.0,0.00,180.00,0.00,180.00
407384,1.0,0.00,269.00,67.25,201.75
407399,90.0,0.00,6.35,0.32,6.03
...,...,...,...,...,...
387799,1.0,16.30,100.15,34.94,65.21
388099,1.0,3.76,23.11,8.06,15.05
388366,1.0,8.15,50.08,17.47,32.61
388367,1.0,8.15,50.08,17.47,32.61


In [111]:
## BE-1-3: provider and the payer disagreement
df2 = df[df['NPHIES_CODE'].isin(['BE-1-3'])][['PROVIDER_DEPARTMENT', 'HIS_INSURANCE_CODE']]
df2[df2['HIS_INSURANCE_CODE']==1494].PROVIDER_DEPARTMENT.unique()

array(['E.N.T. SERVICES - انف واذن وحنجرة',
       'ORTHOPAEDIC SERVICE - العظام', 'NEUROLOGY SERVICES مخ واعصاب',
       'PULMONARY SERVICES - الصدرية', 'OPHTHALMIC SERVICES  - العيون',
       'OBSTERTRIC/GYNEACOLOGIC SERVIC - نساء و ولادة',
       'INTERNAL MEDICINE - الباطنة', 'EMERGENCY ROOM SERVICES - طوارى',
       'DENTAL SERVICES - اسنان',
       'DERMATOLOGY & VENEREOLOGY SERV - جلدية',
       'PAEDIATRIC SERVICES - الاطفال', 'UROLOGY SERVICES -المسالك',
       'CARDIOLOGY SERVICES - القلب', 'NEURO SURGERY - جراحة مخ واعصاب',
       'PLASTIC SURGERY - التجميل',
       'GENERAL SURGERY SERVICES - جراحة عامة',
       'INTENSIVE CARE SERVICES - العناية المركزة',
       'RADIOGRAPHIC INVESTIGATIONS - الأشعة', 'SCREENING DEPARTMENT',
       'MAXILLOFACIAL SURGERY'], dtype=object)

In [110]:
df_2 = df[df['NPHIES_CODE'] != 'BE-1-3'][['PROVIDER_DEPARTMENT', 'HIS_INSURANCE_CODE']]
df_2[df_2['HIS_INSURANCE_CODE']==1494].PROVIDER_DEPARTMENT.unique()
## Notice that the same set of departments exists twice

array(['E.N.T. SERVICES - انف واذن وحنجرة',
       'ORTHOPAEDIC SERVICE - العظام', 'NEUROLOGY SERVICES مخ واعصاب',
       'PULMONARY SERVICES - الصدرية', 'OPHTHALMIC SERVICES  - العيون',
       'OBSTERTRIC/GYNEACOLOGIC SERVIC - نساء و ولادة',
       'INTERNAL MEDICINE - الباطنة', 'EMERGENCY ROOM SERVICES - طوارى',
       'DENTAL SERVICES - اسنان',
       'DERMATOLOGY & VENEREOLOGY SERV - جلدية',
       'PAEDIATRIC SERVICES - الاطفال', 'UROLOGY SERVICES -المسالك',
       'CARDIOLOGY SERVICES - القلب', 'NEURO SURGERY - جراحة مخ واعصاب',
       'PLASTIC SURGERY - التجميل',
       'GENERAL SURGERY SERVICES - جراحة عامة',
       'INTENSIVE CARE SERVICES - العناية المركزة',
       'RADIOGRAPHIC INVESTIGATIONS - الأشعة', 'SCREENING DEPARTMENT',
       'MAXILLOFACIAL SURGERY'], dtype=object)

In [112]:
## BE-1-4: Preauthorization is required and was not obtained (Can't be modeled)
## BE-1-6: Calculation discrepancy from contract (Can't be modeled)
df.columns

Index(['CREATED_BY_USER', 'CREATION_DATE', 'VISIT_ID', 'NET_WITH_VAT',
       'SERVICE_DESCRIPTION', 'QTY', 'LINE_CLAIMED_AMOUNT',
       'LINE_CLAIMED_AMOUNT_SAR', 'CO_PAY', 'CO_INSURANCE',
       'LINE_ITEM_DISCOUNT', 'NET_AMOUNT', 'NET_VAT_AMOUNT',
       'PATIENT_VAT_AMOUNT', 'VAT_PERCENTAGE', 'TREATMENT_TYPE_INDICATOR',
       'SERVICE_TYPE', 'DURATION', 'QTY_STOCKED_UOM', 'UNIT_PRICE',
       'OASIS_IOS_DESCRIPTION', 'SEQUENCE_NO', 'UNIT_PRICE_STOCKED_UOM',
       'UNIT_PRICE_NET', 'DISCOUNT_PERCENTAGE', 'NOTES', 'OUTCOME',
       'APPROVED_QUNATITY', 'REQUEST_DATE', 'VISIT_NO', 'VISIT_DATE',
       'EMERGENCY_INDICATOR', 'PROVIDER_DEPARTMENT_CODE',
       'PROVIDER_DEPARTMENT', 'DOCTOR_SPECIALTY_CODE', 'DOCTOR_CODE',
       'PATIENT_NO', 'EPISODE_NO', 'PATIENT_ID', 'MEMBERSHIP_NO',
       'PATIENT_ID_TYPE', 'PATIENT_NATIONAL_IQAMA_ID', 'PATIENT_DOB',
       'PATIENT_AGE', 'UNIT_OF_AGE', 'PATIENT_NATIONALITY',
       'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'PATIENT_MOBILE_NO'

In [116]:
## BE-1-7: Incorrect billing regime An inpatient claim is submitted as Fee for Service when it should be billed as DRG.
df[df['NPHIES_CODE'] != 'BE-1-7'][['PATIENT_ID_TYPE','TOTAL_DISCOUNT','TOTAL_NET_AMOUNT', 'CLAIM_TYPE']]

,PATIENT_ID_TYPE,TOTAL_DISCOUNT,TOTAL_NET_AMOUNT,CLAIM_TYPE
397551,1.0,2763.6,15488.38,I
397548,1.0,2763.6,15488.38,I
397549,1.0,2763.6,15488.38,I
397550,1.0,2763.6,15488.38,I
397559,1.0,298.6,2352.96,I
...,...,...,...,...
388637,1.0,362.2,893.03,O
388638,1.0,362.2,893.03,O
388639,1.0,362.2,893.03,O
388640,1.0,362.2,893.03,O


In [10]:
## AD-2-4, AD-2-6 : duplicated service or procedure in the same claim, missing the date of claim, and / or claim is submitted after 5 months of the date of service
df[df['NPHIES_CODE'].isin(["AD-2-5","AD-2-6"])][['CREATION_DATE','BATCH_PERIOD_START','BATCH_PERIOD_END', 'SERVICE_DESCRIPTION']]

,CREATION_DATE,BATCH_PERIOD_START,BATCH_PERIOD_END,SERVICE_DESCRIPTION
419923,2024-01-02 16:25:59,2023-12-01,2023-12-31,Dengue IgM/IgG
419924,2024-01-02 16:25:59,2023-12-01,2023-12-31,FARCOLIN 0.5% RESPIRATOR SOLN.
419925,2024-01-02 16:25:59,2023-12-01,2023-12-31,OSELTA 75MG CAPSULES
419926,2024-01-02 16:25:59,2023-12-01,2023-12-31,"Nebulizing system, heated"
419927,2024-01-02 16:25:59,2023-12-01,2023-12-31,Unsp admin of pharmac agent electrolyte
...,...,...,...,...
339407,2024-04-14 16:39:49,2024-03-01,2024-03-31,PARACETOL 10 MG/ML 100 ML BOTTLE (PSI)
340182,2024-04-14 16:46:09,2024-03-01,2024-03-31,Unlisted Code
340183,2024-04-14 16:46:10,2024-03-01,2024-03-31,Quantitative Troponin (Protein) Analysis
396630,2024-04-14 16:46:11,2024-03-01,2024-03-31,"Radiography of chest, 1 view"


In [12]:
df[df['NPHIES_CODE'] == "AD-2-5"][['CREATION_DATE','BATCH_PERIOD_START','BATCH_PERIOD_END', 'SERVICE_DESCRIPTION']]

,CREATION_DATE,BATCH_PERIOD_START,BATCH_PERIOD_END,SERVICE_DESCRIPTION
420802,2024-01-02 19:50:02,2023-12-01,2023-12-31,FLUCA OPTHALMIC SUSP.
420803,2024-01-02 19:50:02,2023-12-01,2023-12-31,Consultant Consultation
420801,2024-01-02 19:50:02,2023-12-01,2023-12-31,TOBRADEX EYE OINT
414051,2024-01-03 17:40:06,2023-12-01,2023-12-31,SOCLAV 1 G/125 MG 16 SACHET
414050,2024-01-03 17:40:06,2023-12-01,2023-12-31,SOLPADEINE SOLUBLE TABLET
...,...,...,...,...
291711,2024-04-06 05:54:25,2024-03-01,2024-03-31,SALURIN 20MG-2ML AMPOULES
291712,2024-04-06 05:54:25,2024-03-01,2024-03-31,SALURIN 20MG-2ML AMPOULES
291714,2024-04-06 05:54:25,2024-03-01,2024-03-31,EPINOR 4 MG / 4 ML 5 AMP
291706,2024-04-06 05:54:25,2024-03-01,2024-03-31,Unlisted Procedure Code


In [19]:
df[df['NPHIES_CODE'] == "AD-2-6"][['CREATION_DATE','BATCH_PERIOD_START','BATCH_PERIOD_END', 'SERVICE_DESCRIPTION']]

,CREATION_DATE,BATCH_PERIOD_START,BATCH_PERIOD_END,SERVICE_DESCRIPTION
419923,2024-01-02 16:25:59,2023-12-01,2023-12-31,Dengue IgM/IgG
419924,2024-01-02 16:25:59,2023-12-01,2023-12-31,FARCOLIN 0.5% RESPIRATOR SOLN.
419925,2024-01-02 16:25:59,2023-12-01,2023-12-31,OSELTA 75MG CAPSULES
419926,2024-01-02 16:25:59,2023-12-01,2023-12-31,"Nebulizing system, heated"
419927,2024-01-02 16:25:59,2023-12-01,2023-12-31,Unsp admin of pharmac agent electrolyte
...,...,...,...,...
339407,2024-04-14 16:39:49,2024-03-01,2024-03-31,PARACETOL 10 MG/ML 100 ML BOTTLE (PSI)
340182,2024-04-14 16:46:09,2024-03-01,2024-03-31,Unlisted Code
340183,2024-04-14 16:46:10,2024-03-01,2024-03-31,Quantitative Troponin (Protein) Analysis
396630,2024-04-14 16:46:11,2024-03-01,2024-03-31,"Radiography of chest, 1 view"


In [18]:
df.CREATION_DATE.sort_values().iloc[-1]

Timestamp('2024-04-30 12:13:09')

In [9]:
df[~df['NPHIES_CODE'].isin(["AD-2-4","AD-2-5","AD-2-6"])][['CREATION_DATE','BATCH_PERIOD_START','BATCH_PERIOD_END', 'SERVICE_DESCRIPTION']]

,CREATION_DATE,BATCH_PERIOD_START,BATCH_PERIOD_END,SERVICE_DESCRIPTION
397551,2024-01-01 09:58:15,2023-12-01,2023-12-10,Routine preoperative anaes assessment
397548,2024-01-01 09:58:15,2023-12-01,2023-12-10,Cross Matching
397549,2024-01-01 09:58:15,2023-12-01,2023-12-10,TRANDATE INJ. 100 MG.-AMP.20ML
397550,2024-01-01 09:58:15,2023-12-01,2023-12-10,Elective classical caesarean section
397559,2024-01-01 10:00:43,2023-12-01,2023-12-10,"Hypodermic needle, single-use"
...,...,...,...,...
388637,2024-04-30 12:13:09,2024-04-01,2024-04-10,Peripheral intravenous cannula
388638,2024-04-30 12:13:09,2024-04-01,2024-04-10,Ultrasound of pelvis
388639,2024-04-30 12:13:09,2024-04-01,2024-04-10,Urinalysis Test
388640,2024-04-30 12:13:09,2024-04-01,2024-04-10,PANADREX 500 MG 24 TAB


In [7]:
df['NPHIES_CODE'].value_counts()

NPHIES_CODE
MN-1-1     17806
CV-1-1     12178
SE-1-6     10042
BE-1-1      7177
BE-1-7      7159
BE-1-3      4530
BE-1-6      4493
BE-1-4      2604
SE-1-8      1607
SE-1-9      1501
CV-1-3      1059
AD-2-4       759
AD-2-5       735
BE-1-5       500
CV-1-4       487
CV-3-2       441
AD-2-6       365
AD-3-2       326
CV-3-5       280
AD-1-6       233
CV-1-2       223
CV-1-8       211
SE-1-2       142
AD-3-3       107
AD-1-3        77
AD-1-7        54
CV-2-1        44
CV-3-1        31
AD-1-4        24
CV-1-9        20
AD-3-5        19
AD-3-7        15
BE-1-10        9
BE-1-2         9
MN-1-2         6
CV-1-10        5
AD-3-8         3
SE-1-10        1
SE-1-5         1
SE-1-7         1
Name: count, dtype: int64

In [55]:
REVS = ['VISIT_ID','DOCTOR_CODE','TECHNICAL_ISSUE']
df[df['TECHNICAL_ISSUE']=='DOCTOR_CODE'][REVS]

,VISIT_ID,DOCTOR_CODE,TECHNICAL_ISSUE


In [15]:
df['OUTCOME'].value_counts()

OUTCOME
APPROVED    91507
REJECTED    69008
PARTIAL     11476
Name: count, dtype: int64

## HJH Dataset
loaded on 23/5/2024

In [3]:
df_original = get_cashed_input()

In [4]:
df_original['CREATION_DATE'].sort_values(ascending=False)

388640   2024-04-30 12:13:09
388639   2024-04-30 12:13:09
388633   2024-04-30 12:13:09
388634   2024-04-30 12:13:09
388635   2024-04-30 12:13:09
                 ...        
397544   2024-01-01 09:58:15
397545   2024-01-01 09:58:15
397546   2024-01-01 09:58:15
397547   2024-01-01 09:58:15
397549   2024-01-01 09:58:15
Name: CREATION_DATE, Length: 488833, dtype: datetime64[ns]

In [5]:
df_original.VISIT_ID.value_counts()

VISIT_ID
220020    1819
202392    1529
203244    1529
201986    1529
202394    1529
          ... 
198125       1
228482       1
210011       1
228691       1
231136       1
Name: count, Length: 42771, dtype: int64

In [6]:
preprocessing = MergedDataPreprocessing(df_original)

In [7]:
for col in df_original.columns:
    print(col,":",df_original[col].dtype)

CREATED_BY_USER : int64
CREATION_DATE : datetime64[ns]
VISIT_ID : int64
NET_WITH_VAT : float64
SERVICE_DESCRIPTION : object
QTY : float64
LINE_CLAIMED_AMOUNT : float64
LINE_CLAIMED_AMOUNT_SAR : float64
CO_PAY : float64
CO_INSURANCE : float64
LINE_ITEM_DISCOUNT : float64
NET_AMOUNT : float64
NET_VAT_AMOUNT : float64
PATIENT_VAT_AMOUNT : float64
VAT_PERCENTAGE : float64
TREATMENT_TYPE_INDICATOR : int64
SERVICE_TYPE : object
DURATION : float64
QTY_STOCKED_UOM : int64
UNIT_PRICE : float64
OASIS_IOS_DESCRIPTION : object
SEQUENCE_NO : int64
UNIT_PRICE_STOCKED_UOM : float64
UNIT_PRICE_NET : float64
DISCOUNT_PERCENTAGE : float64
NOTES : object
OUTCOME : object
APPROVED_QUNATITY : float64
REQUEST_DATE : datetime64[ns]
VISIT_NO : object
VISIT_DATE : datetime64[ns]
EMERGENCY_INDICATOR : object
PROVIDER_DEPARTMENT_CODE : int64
PROVIDER_DEPARTMENT : object
DOCTOR_SPECIALTY_CODE : float64
DOCTOR_CODE : int64
PATIENT_NO : int64
EPISODE_NO : int64
PATIENT_ID : int64
MEMBERSHIP_NO : object
PATIENT_ID_T

In [8]:
df_original.isna().sum()

CREATED_BY_USER            0
CREATION_DATE              0
VISIT_ID                   0
NET_WITH_VAT               0
SERVICE_DESCRIPTION        0
                       ...  
BATCH_PERIOD_START         0
BATCH_PERIOD_END           0
Episode_Key            31109
ICD10                  31109
Diagnosis_Key          31109
Length: 96, dtype: int64

In [9]:
df_original.SUBMIT_CLAIM_OUTCOME.value_counts()

SUBMIT_CLAIM_OUTCOME
PENDED    241830
QUEUED     70352
ERROR      64160
Name: count, dtype: int64

In [10]:
df_prep = preprocessing.columns_prep()

C:\AI_Projects\Claims_Rejection_2024\Claims_Rejection\src\data_local.py:203: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[column] = self.df[column].replace(column_encoding)
C:\AI_Projects\Claims_Rejection_2024\Claims_Rejection\src\data_local.py:208: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
C:\AI_Projects\Claims_Rejection_2024\Claims_Rejection\src\data_local.py:214: UserWarning: Pandas doesn't allow columns to be created via a new attribute nam

In [11]:
df_prep

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,Episode_Key,ICD10,Diagnosis_Key,PatientAgeRange
0,9521,2024-04-04 22:57:57,230089,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,...,0,H,2024-03-28 00:00:00,2024-04-11 00:00:00,2024-03-01,2024-03-31,1_929362-34,Z36,1_10198603,6
1,9521,2024-04-04 23:02:21,230117,202.40,TSH,1.0,220.00,220.00,0.00,220.00,...,0,H,2024-03-12 00:00:00,2024-03-26 00:00:00,2024-03-01,2024-03-31,1_1083147-3,R22.1,1_10171516,5
2,9521,2024-04-04 23:08:38,230160,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,...,0,H,2024-03-04 00:00:00,2024-03-18 00:00:00,2024-03-01,2024-03-31,1_1339075-6,R53,1_1111232092,6
3,9521,2024-04-04 22:26:08,229872,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,...,0,H,2024-03-10 00:00:00,2024-03-24 00:00:00,2024-03-01,2024-03-31,1_656928-60,R10,1_1111231597,6
4,9521,2024-04-04 22:34:22,229913,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,...,0,H,2024-03-18 00:00:00,2024-04-01 00:00:00,2024-03-01,2024-03-31,1_1019628-25,R00.1,1_10171441,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,9593,2024-01-23 17:20:54,204110,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,...,0,IQ,2023-12-24 10:28:07,2023-12-28 12:07:18,2023-12-01,2023-12-31,1_1348956-2,I24.9,1_10163946,8
488829,10962,2024-01-28 01:01:05,204377,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,...,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,1_1349595-2,R07.4,1_110217036,7
488830,10962,2024-01-28 01:01:05,204377,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,...,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,1_1349595-2,R07.4,1_110217036,7
488831,10962,2024-01-28 01:01:05,204377,1.79,SALURIN 20MG-2ML AMPOULES,1.0,1.93,1.93,0.00,1.93,...,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,1_1349595-2,R07.4,1_110217036,7


In [12]:
df_train, df_test = preprocessing.train_test_split(test_size=0.2,random_state=6)

In [13]:
len(df_test)/ len(df_train)

0.2531547822058057

In [14]:
ls1 = list(df_train.sort_values(by='VISIT_ID')['VISIT_ID'])
ls2 = list(df_test.sort_values(by='VISIT_ID')['VISIT_ID'])
intersection = list(set(ls1) & set(ls2))
intersection ## to find the repeated transaction id between them

[]

In [15]:
len(df_train) / len(df_test)

3.9501525165226234

In [16]:
df_train.to_parquet('data/HJH/prq/train.parquet')
df_test.to_parquet('data/HJH/prq/test.parquet')